## 1.背景介绍

本实验主要研究 Megatron-LM 分布式训练框架下的张量并行训练方法。

## 2.实验目的
了解张量并行的基本思想，利用 Megatron-LM 并行训练框架，完成两卡张量并行训练。

## 3.硬件要求

两张 GPU（4090、V100、A100等）。


## 4.技术原理

### 张量并行

为了解决朴素数据并行显存节省效率不高，朴素流水线并行计算效率不高的问题，张量并行对模型纵向切分，每张 GPU 保存一层的部分权重，在运算时，输入张量在每一张 GPU 上分别计算，然后通过 GPU 之间的通信聚合起来。

相较于朴素的流水线并行，张量并行每一次计算均有多张 GPU 参与。

相较于朴素的数据并行，张量并行每一张 GPU 保存部分模型权重。

## 5.实验流程

### 环境配置



In [1]:
!git clone https://github.com/NVIDIA/Megatron-LM.git

Cloning into 'Megatron-LM'...
remote: Enumerating objects: 48313, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 48313 (delta 162), reused 155 (delta 128), pack-reused 48064 (from 2)
Receiving objects: 100% (48313/48313), 18.04 MiB | 306.00 KiB/s, done.
Resolving deltas: 100% (33793/33793), done.


In [9]:
!pip install torch einops pybind11

  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)


### 5.1. 包导入以及通信组设置

In [28]:

import os
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from functools import partial
from pathlib import Path

import sys
sys.path.append("./Megatron-LM")


from megatron.core import parallel_state
from megatron.core import dist_checkpointing
from megatron.core.pipeline_parallel.schedules import get_forward_backward_func
from megatron.core.tensor_parallel.random import model_parallel_cuda_manual_seed
from megatron.core.transformer.transformer_config import TransformerConfig
from megatron.core.models.gpt.gpt_model import GPTModel
from megatron.core.models.gpt.gpt_layer_specs import get_gpt_layer_local_spec
from megatron.core.datasets.utils import compile_helpers 
from megatron.core.datasets.blended_megatron_dataset_builder import BlendedMegatronDatasetBuilder
from megatron.core.datasets.gpt_dataset import GPTDatasetConfig, MockGPTDataset
from megatron.training.tokenizer.tokenizer import _NullTokenizer


_SEQUENCE_LENGTH = 64


def initialize_distributed(tensor_model_parallel_size=1, pipeline_model_parallel_size=1):
    parallel_state.destroy_model_parallel()

    # Torch setup for distributed training
    rank = int(os.environ['LOCAL_RANK'])
    world_size = torch.cuda.device_count()
    torch.cuda.set_device(rank)
    torch.distributed.init_process_group(world_size=world_size, rank=rank)

    # Megatron core distributed training initialization
    parallel_state.initialize_model_parallel(tensor_model_parallel_size, pipeline_model_parallel_size)

/nvme/zhanglantian/project/DistributedTrainingDemos/demo4/./Megatron-LM/megatron/core/transformer/multi_token_prediction.py:60: UserWarning: Apex is not installed. Falling back to Torch Norm
  warnings.warn('Apex is not installed. Falling back to Torch Norm')
/nvme/zhanglantian/project/DistributedTrainingDemos/demo4/./Megatron-LM/megatron/core/models/gpt/gpt_layer_specs.py:64: UserWarning: Apex is not installed. Falling back to Torch Norm
  warnings.warn('Apex is not installed. Falling back to Torch Norm')
/nvme/zhanglantian/project/DistributedTrainingDemos/demo4/./Megatron-LM/megatron/core/models/retro/encoder_spec.py:47: UserWarning: Apex is not installed. Falling back to Torch Norm
  warnings.warn(f'Apex is not installed. Falling back to Torch Norm')
/nvme/zhanglantian/project/DistributedTrainingDemos/demo4/./Megatron-LM/megatron/core/models/retro/decoder_spec.py:39: UserWarning: Apex is not installed. Falling back to Torch Norm
  warnings.warn(f'Apex is not installed. Falling back 

### 5.2. transformer模型定义

In [30]:
def model_provider():
    """Build the model."""

    transformer_config = TransformerConfig(
        num_layers=10, 
        hidden_size=512, 
        num_attention_heads=4, 
        use_cpu_initialization=True, 
        pipeline_dtype=torch.float32,
    )

    gpt_model = GPTModel(
        config=transformer_config, 
        transformer_layer_spec=get_gpt_layer_local_spec(), 
        vocab_size=100, 
        max_sequence_length=_SEQUENCE_LENGTH,
    )

    return gpt_model

### 5.3.数据集定义

In [31]:
def get_train_data_iterator():
    if torch.distributed.is_available() and torch.distributed.is_initialized():
        if torch.distributed.get_rank() == 0:
            compile_helpers()
        torch.distributed.barrier()
    else:
        compile_helpers()

    config = GPTDatasetConfig(
        random_seed=0,
        sequence_length=_SEQUENCE_LENGTH,
        reset_position_ids=False,
        reset_attention_mask=False,
        eod_mask_loss=False,
        tokenizer=_NullTokenizer(vocab_size=_SEQUENCE_LENGTH),
    )

    datasets = BlendedMegatronDatasetBuilder(
        MockGPTDataset, [1000, None, None], lambda: True, config
    ).build()

    train_dataloader = DataLoader(datasets[0], batch_size=128, shuffle=True)

    train_iterator = iter(train_dataloader)

    return train_iterator

### 5.4.训练核心代码实现

In [33]:
def forward_step_func(data_iterator, model):

    def loss_func(loss_mask: torch.Tensor, output_tensor: torch.Tensor):

        losses = output_tensor.float()
        loss_mask = loss_mask.view(-1).float()
        loss = torch.sum(losses.view(-1) * loss_mask) / loss_mask.sum()
        # If you have data parallel reduce loss across data parallel groups.
        # If pipeline parallel, loss computation is done only in last stage.

        return loss, {'lm loss': loss}

    data = next(data_iterator)
    tokens = data['tokens'].to(device)
    attention_mask = data['attention_mask'].to(device)
    position_ids = data['position_ids'].to(device)
    labels = data['labels'].to(device)
    loss_mask = data['loss_mask'].to(device)

    output_tensor = model(tokens, position_ids, attention_mask,
                          labels=labels)

    return output_tensor, partial(loss_func, loss_mask)

def save_distributed_checkpoint(checkpoint_path, gpt_model):
    sharded_state_dict = gpt_model.sharded_state_dict(prefix='')
    dist_checkpointing.save(sharded_state_dict=sharded_state_dict, checkpoint_dir=checkpoint_path)

def load_distributed_checkpoint(checkpoint_path, gpt_model):
    sharded_state_dict=gpt_model.sharded_state_dict(prefix='')
    checkpoint = dist_checkpointing.load(sharded_state_dict=sharded_state_dict, checkpoint_dir=checkpoint_path)
    gpt_model.load_state_dict(checkpoint)
    return gpt_model


In [34]:
!pwd
!CUDA_VISIBLE_DEVICES=6,7 torchrun --nproc-per-node 2 train.py

/nvme/zhanglantian/project/DistributedTrainingDemos/demo4
W0325 15:59:32.875000 62447 site-packages/torch/distributed/run.py:793] 
W0325 15:59:32.875000 62447 site-packages/torch/distributed/run.py:793] *****************************************
W0325 15:59:32.875000 62447 site-packages/torch/distributed/run.py:793] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0325 15:59:32.875000 62447 site-packages/torch/distributed/run.py:793] *****************************************
/nvme/zhanglantian/project/DistributedTrainingDemos/demo4/./Megatron-LM/megatron/core/transformer/multi_token_prediction.py:60: UserWarning: Apex is not installed. Falling back to Torch Norm
  warnings.warn('Apex is not installed. Falling back to Torch Norm')
/nvme/zhanglantian/project/DistributedTrainingDemos/demo4/./Megatron-LM/megatron/core/models/gpt/gp